In [1]:
import optuna
import pickle
import plotly 
from src.models.hyperopt_emos import Objective

2024-06-07 19:39:38.059551: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-07 19:39:38.062212: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-07 19:39:38.095884: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-07 19:39:54.081934: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
features_names = ['wind_speed', 'press', 'kinetic', 'humid', 'geopot']

features_names_dict = {name: 1 for name in features_names}

features_names_dict['wind_speed'] = 15

objectives = ['twCRPS12']


In [33]:
#study_name = "crps_obj_CRPS_twCRPS12_MOTPE.pkl"
study_name = "crps_obj_twCRPS12_TPE.pkl"
filepath = '/net/pc200239/nobackup/users/hakvoort/hyperopt/hyperopt_emos/'

# an optuna study
study = pickle.load(open(filepath + study_name, 'rb'))

In [34]:
study2 = optuna.load_study(study_name='twcrps_obj_CRPS_twCRPS12_MOTPE', storage=f'sqlite:///{filepath}/study.db')
study2.set_metric_names(['CRPS', 'twCRPS12'])

/tmp/ipykernel_2400814/3646120845.py:2: ExperimentalWarning:

set_metric_names is experimental (supported from v3.2.0). The interface can change in the future.



In [35]:
#give hyperparameter importances

optuna.visualization.plot_param_importances(study2).show()

In [36]:
print(len(study2.trials))

177


In [15]:
optuna.visualization.plot_pareto_front(study2)

In [37]:
df = study2.trials_dataframe()

new_column_names = {
    'params_chain_function_mean': 'cf_mean',
    'params_chain_function_std': 'cf_std',
    'params_chain_function_constant': 'cf_constant',
    'values_CRPS': 'CRPS',
    'values_twCRPS12': 'twCRPS12',
    'params_forecast_distribution': 'forecast_distribution',
    'params_distribution_1': 'distribution_1',
    'params_distribution_2': 'distribution_2',
    'params_epochs': 'epochs',
    'params_learning_rate': 'learning_rate',
    'params_batch_size': 'batch_size',
    'params_optimizer': 'optimizer'
}

df.rename(columns=new_column_names, inplace=True)

print(df.columns)

Index(['number', 'CRPS', 'twCRPS12', 'datetime_start', 'datetime_complete',
       'duration', 'batch_size', 'cf_constant', 'cf_mean', 'cf_std',
       'distribution_1', 'distribution_2', 'epochs', 'forecast_distribution',
       'learning_rate', 'optimizer', 'params_perform_batching', 'state'],
      dtype='object')


In [38]:
top_CRPS = df.sort_values('CRPS').head(12)

In [39]:
top_twCRPS12 = df.sort_values('twCRPS12').head(8)

In [40]:

pareto_front = study2.best_trials
best_runs = [trial.number for trial in pareto_front]

pareto_trials = df['number'].isin(best_runs)

my_run = df[pareto_trials]


In [41]:
params = ['cf_mean',
           'cf_std', 
           'cf_constant',
             'forecast_distribution', 
             'CRPS',
             'distribution_1',
              'distribution_2',
             'twCRPS12',
             ]

In [42]:
display(top_CRPS[params])

,cf_mean,cf_std,cf_constant,forecast_distribution,CRPS,distribution_1,distribution_2,twCRPS12
171,-3.828661,1.485626,0.000203,distr_mixture_linear,2.695017,distr_trunc_normal,distr_gev,0.178157
160,-2.897627,3.120845,0.000129,distr_mixture_linear,2.696010,distr_trunc_normal,distr_gev,0.178914
164,-1.096566,1.823862,0.000185,distr_mixture_linear,2.698436,distr_trunc_normal,distr_gev,0.178385
73,8.145216,7.913398,0.000104,distr_mixture_linear,2.701168,distr_trunc_normal,distr_gev,0.178784
168,-3.234054,3.584374,0.000153,distr_mixture_linear,2.702137,distr_trunc_normal,distr_gev,0.179253
162,1.424215,1.919470,0.000244,distr_mixture_linear,2.702692,distr_trunc_normal,distr_gev,0.180478
77,11.201177,7.358788,0.000272,distr_mixture_linear,2.703183,distr_trunc_normal,distr_gev,0.179080
161,-4.099851,3.046601,0.000133,distr_mixture_linear,2.703959,distr_trunc_normal,distr_gev,0.180210
156,-0.128789,0.005772,0.986325,distr_mixture,2.704629,distr_log_normal,distr_trunc_normal,0.180394
170,-2.671768,1.681165,0.000186,distr_mixture_linear,2.705825,distr_trunc_normal,distr_gev,0.180701


In [43]:
display(top_twCRPS12[params])

,cf_mean,cf_std,cf_constant,forecast_distribution,CRPS,distribution_1,distribution_2,twCRPS12
60,10.816164,1.785932,0.000210,distr_trunc_normal,2.845130,NaN,NaN,0.176170
166,11.197616,1.662136,0.000015,distr_trunc_normal,2.880362,NaN,NaN,0.176460
111,10.476087,0.000453,0.000302,distr_trunc_normal,3.029498,NaN,NaN,0.176803
114,10.971715,0.960693,0.000144,distr_mixture,2.976686,distr_log_normal,distr_trunc_normal,0.177031
54,12.199493,1.329734,0.000066,distr_trunc_normal,3.031583,NaN,NaN,0.177050
120,10.775414,1.627839,0.000229,distr_trunc_normal,2.859386,NaN,NaN,0.177171
172,10.817245,1.401593,0.000381,distr_trunc_normal,2.876785,NaN,NaN,0.177331
50,11.881624,1.164704,0.000083,distr_trunc_normal,3.035117,NaN,NaN,0.177576


In [20]:
my_run = my_run.sort_values('CRPS')
display(my_run[params])

,cf_mean,cf_std,cf_constant,forecast_distribution,CRPS,twCRPS12
171,-3.828661,1.485626,0.000203,distr_mixture_linear,2.695017,0.178157
75,9.197862,2.935806,0.000895,distr_mixture_linear,2.733251,0.177762
133,10.426935,2.914051,0.000202,distr_trunc_normal,2.757171,0.177633
60,10.816164,1.785932,0.000210,distr_trunc_normal,2.845130,0.176170
